# NNodely Documentation - Inference

Here are shown all the modalities in which you can make inference with the model.

In [1]:
# uncomment the command below to install the nnodely package
#!pip install nnodely

from nnodely import *
from nnodely.relation import NeuObj

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>-- nnodely_v1.3.1 --<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<


## Simple inference

To make inference, just call the nnodely framework class by passing the dictionary of inputs. The model will execute a forward pass and return the outputs of the network.

It is mandatory to call the inference with at least 1 possible prediction window for each declared inputs. the framework will predict the maximum possible window given the data

In [2]:
NeuObj.clearNames()
## Model definition
x = Input('x')
F = Input('F')

px = Parameter('px', tw=0.5, values=[[1.],[1.],[1.],[1.],[1.],[1.],[1.],[1.],[1.],[1.]])
pf = Parameter('pf', sw=1, values=[[1.]])
next_x = Fir(W=px)(x.tw(0.5))+Fir(W=pf)(F.last())

out = Output('next_x', next_x)

model = Modely()
model.addModel('model',[out])
model.neuralizeModel(0.05)

================================ nnodely Model =================================
{'Constants': {},
 'Functions': {},
 'Info': {'SampleTime': 0.05,
          'nnodely_version': '1.3.1',
          'ns': [10, 0],
          'ntot': 10,
          'num_parameters': 11},
 'Inputs': {'F': {'dim': 1,
                  'ns': [1, 0],
                  'ntot': 1,
                  'sw': [-1, 0],
                  'tw': [0, 0]},
            'x': {'dim': 1,
                  'ns': [10, 0],
                  'ntot': 10,
                  'sw': [0, 0],
                  'tw': [-0.5, 0]}},
 'Minimizers': {},
 'Models': 'model',
 'Outputs': {'next_x': 'Add6'},
 'Parameters': {'pf': {'dim': 1, 'sw': 1, 'values': [[1.0]]},
                'px': {'dim': 1,
                       'tw': 0.5,
                       'values': [[1.0],
                                  [1.0],
                                  [1.0],
                                  [1.0],
                                  [1.0],
               

In [3]:
## Inference
results = model(inputs={'F':[[9]],'x':[[1],[2],[3],[4],[5],[6],[7],[8],[9],[10],[11]]})
print(results)

[__call__] Different number of samples between inputs [MAX 2 = 2; MIN 1 = 1]
{'next_x': [64.0]}


In [4]:
results = model(inputs={'F':[[5],[4],[9]],'x':[[1],[2],[3],[4],[5],[6],[7],[8],[9],[10],[11],[12],[13]]})
print(results)

[__call__] Different number of samples between inputs [MAX 4 = 4; MIN 3 = 3]
{'next_x': [60.0, 69.0, 84.0]}


## Inference with a sampled data

Using the option sampled=True the inference window is left to the user to define by passing every sampling input.

In [5]:
results = model(inputs={'F':[[5],[2]],'x':[[1,2,3,4,5,6,7,8,9,10],[12,13,14,15,16,17,18,19,20,21]]}, sampled=True)
print(results)

{'next_x': [62.0, []]}


## Closed Loop and Connect during Inference

During inference time you can define closed loop and connect relations between the variables.

In [9]:
NeuObj.clearNames()
input1 = Input('in1')
W = Parameter('W', sw=3, values=[[1], [2], [3]])
out = Output('out',Fir(W=W)(input1.sw(3)))

model = Modely(visualizer=TextVisualizer(), seed=42)
model.addModel('model', [out])
model.neuralizeModel()

result = model({'in1': [1, 2, 3]}, closed_loop={'in1':'out'})
print(result)

================================ nnodely Model =================================
{'Constants': {},
 'Functions': {},
 'Info': {'SampleTime': 1,
          'nnodely_version': '1.3.1',
          'ns': [3, 0],
          'ntot': 3,
          'num_parameters': 3},
 'Inputs': {'in1': {'dim': 1,
                    'ns': [3, 0],
                    'ntot': 3,
                    'sw': [-3, 0],
                    'tw': [0, 0]}},
 'Minimizers': {},
 'Models': 'model',
 'Outputs': {'out': 'Fir15'},
 'Parameters': {'W': {'dim': 1, 'sw': 3, 'values': [[1.0], [2.0], [3.0]]}},
 'Relations': {'Fir15': ['Fir', ['SamplePart14'], 'W', None, 0],
               'SamplePart14': ['SamplePart', ['in1'], -1, [-3, 0]]},
 'States': {}}
{'out': [14.0]}


In [10]:
input2 = Input('in2')
K = Parameter('K', sw=3, values=[[1], [2], [3]])
out2 = Output('out2',Fir(W=K)(input2.sw(3)))

model.addModel('model2', [out2])
model.neuralizeModel()

result = model(inputs={'in1': [1, 2, 3]}, connect={'in2':'out'})
print(result)

================================ nnodely Model =================================
{'Constants': {},
 'Functions': {},
 'Info': {'SampleTime': 1,
          'nnodely_version': '1.3.1',
          'ns': [3, 0],
          'ntot': 3,
          'num_parameters': 6},
 'Inputs': {'in1': {'dim': 1,
                    'ns': [3, 0],
                    'ntot': 3,
                    'sw': [-3, 0],
                    'tw': [0, 0]},
            'in2': {'dim': 1,
                    'ns': [3, 0],
                    'ntot': 3,
                    'sw': [-3, 0],
                    'tw': [0, 0]}},
 'Minimizers': {},
 'Models': {'model': {'Constants': [],
                      'Inputs': ['in1'],
                      'Outputs': ['out'],
                      'Parameters': ['W'],
                      'States': []},
            'model2': {'Constants': [],
                       'Inputs': ['in2'],
                       'Outputs': ['out2'],
                       'Parameters': ['K'],
                   

## Recurrent inference with prediction samples

to do multiple recurrent inference with the state variables just decide the number of samples to predict using the attribute 'prediction_samples'.

it can also be 'auto' meaning that the recurrent inference will continue until there are inputs available.

In [11]:
result = model({'in1': [1, 2, 3, 4, 5]},closed_loop={'in1':'out'}, prediction_samples=3)
print(result)

[__call__] Inputs not provided: ['in2']. Autofilling with zeros..
{'out2': [0.0, 0.0, 0.0], 'out': [14.0, 50.0, 181.0]}


## Recurrent inference with number of samples

By specifying the number of samples when doing inference with states variables

The number of samples will specify for how many steps the inference must go on. This can be done only when using states variables 

In [13]:
result = model({'in1': [1, 2, 3, 4, 5]},closed_loop={'in1':'out'}, prediction_samples=3, num_of_samples = 5)
print(result)

[__call__] The variable in1 is filled with 2 samples equal to zeros.
[__call__] Inputs not provided: ['in2']. Autofilling with zeros..
{'out2': [0.0, 0.0, 0.0, 0.0, 0.0], 'out': [14.0, 50.0, 181.0, 657.0, 5.0]}
